In [72]:
import pandas as pd 
import numpy as np

import sbmlcore
import pytest

import warnings
warnings.filterwarnings('ignore')

Load WHO data

In [73]:
WHO_data = pd.read_csv('data/NC_000962.3_WHO-UCN-GTB-PCI-2021.7_v1.0_GARC1_RFUS.csv')
WHO_data.MUTATION
result = ['gyr' in row for row in WHO_data['MUTATION']]
WHO_data = WHO_data[result & (WHO_data['PREDICTION'] != 'F') & (WHO_data['PREDICTION'] != 'U')]
WHO_data

,Unnamed: 0.1,GENBANK_REFERENCE,CATALOGUE_NAME,CATALOGUE_VERSION,CATALOGUE_GRAMMAR,PREDICTION_VALUES,DRUG,MUTATION,PREDICTION,SOURCE,EVIDENCE,OTHER,Unnamed: 0
1178,645,NC_000962.3,WHO-UCN-GTB-PCI-2021.7,1.0,GARC1,RFUS,LEV,gyrA@A90V,R,{},"{""Present_SOLO_R"": 590, ""Present_SOLO_SR"": 658...","{""FINAL_CONFIDENCE_GRADING"": ""1) Assoc w R""}",NaN
1179,646,NC_000962.3,WHO-UCN-GTB-PCI-2021.7,1.0,GARC1,RFUS,MXF,gyrA@A90V,R,{},"{""Present_SOLO_R"": 253, ""Present_SOLO_SR"": 598...","{""FINAL_CONFIDENCE_GRADING"": ""1) Assoc w R""}",NaN
1180,647,NC_000962.3,WHO-UCN-GTB-PCI-2021.7,1.0,GARC1,RFUS,LEV,gyrA@D94A,R,{},"{""Present_SOLO_R"": 181, ""Present_SOLO_SR"": 207...","{""FINAL_CONFIDENCE_GRADING"": ""1) Assoc w R""}",NaN
1181,648,NC_000962.3,WHO-UCN-GTB-PCI-2021.7,1.0,GARC1,RFUS,MXF,gyrA@D94A,R,{},"{""Present_SOLO_R"": 89, ""Present_SOLO_SR"": 187,...","{""FINAL_CONFIDENCE_GRADING"": ""1) Assoc w R""}",NaN
1182,649,NC_000962.3,WHO-UCN-GTB-PCI-2021.7,1.0,GARC1,RFUS,LEV,gyrA@D94G,R,{},"{""Present_SOLO_R"": 1133, ""Present_SOLO_SR"": 11...","{""FINAL_CONFIDENCE_GRADING"": ""1) Assoc w R""}",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,731,NC_000962.3,WHO-UCN-GTB-PCI-2021.7,1.0,GARC1,RFUS,MXF,gyrB@c-165t,S,{},"{""Present_SOLO_R"": 2, ""Present_SOLO_SR"": 333, ...","{""FINAL_CONFIDENCE_GRADING"": ""5) Not assoc w R""}",NaN
2126,1593,NC_000962.3,WHO-UCN-GTB-PCI-2021.7,1.0,GARC1,RFUS,LEV,gyrA@*=,S,{},{},{},NaN
2131,1598,NC_000962.3,WHO-UCN-GTB-PCI-2021.7,1.0,GARC1,RFUS,MXF,gyrA@*=,S,{},{},{},NaN
2136,1603,NC_000962.3,WHO-UCN-GTB-PCI-2021.7,1.0,GARC1,RFUS,LEV,gyrB@*=,S,{},{},{},NaN


Load MIC (continous, but also binary available) phenotypic data from CRyPTIC table

In [74]:
PHENOTYPES = pd.read_pickle('data/UKMYC_PHENOTYPES.pkl.gz')
PHENOTYPES.reset_index(inplace = True)
PHENOTYPES.set_index('UNIQUEID', inplace = True)
PHENOTYPES = PHENOTYPES[(PHENOTYPES.DRUG == 'LEV') | (PHENOTYPES.DRUG == 'MXF')]
PHENOTYPES

,DRUG,PLATEDESIGN,BELONGS_GPI,SITEID,DILUTION,PHENOTYPE_QUALITY,READINGDAY,PRIMARY_DILUTION,PRIMARY_METHOD,AMYGDA_DILUTION,BASHTHEBUG_DILUTION,BASHTHEBUGPRO_DILUTION,PHENOTYPE_DESCRIPTION,BASHTHEBUG_NUMBER_CLASSIFICATIONS,BASHTHEBUGPRO_NUMBER_CLASSIFICATIONS,MIC,LOG2MIC,BINARY_PHENOTYPE
UNIQUEID,,,,,,,,,,,,,,,,,,
site.06.subj.06TB_1032.lab.06MIL2037.iso.1,MXF,UKMYC6,True,06,3.0,HIGH,14,3.0,VZ,3.0,8.0,NaN,"VZ,IM AGREE",11.0,NaN,0.25,-2.00,S
site.06.subj.06TB_0404.lab.06MIL1373.iso.1,MXF,UKMYC6,True,06,3.0,HIGH,21,3.0,VZ,3.0,8.0,NaN,"VZ,IM AGREE",11.0,NaN,0.25,-2.00,S
site.02.subj.1518.lab.2013102308.iso.1,LEV,UKMYC6,False,02,3.0,LOW,14,3.0,VZ,2.0,4.0,3.0,ALL DISAGREE,11.0,3.0,0.5,-1.00,S
site.06.subj.SSM_0197-R.lab.06MIL0117.iso.1,MXF,UKMYC5,True,06,1.0,HIGH,14,1.0,VZ,1.0,8.0,NaN,"VZ,IM AGREE",11.0,NaN,<=0.06,-4.06,S
site.06.subj.06TB_1545.lab.06MIL2824.iso.1,MXF,UKMYC6,True,06,2.0,LOW,14,2.0,VZ,4.0,8.0,4.0,ALL DISAGREE,11.0,3.0,0.12,-3.06,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
site.02.subj.0016.lab.2014231005.iso.1,LEV,UKMYC5,True,02,2.0,HIGH,14,2.0,VZ,2.0,2.0,NaN,"VZ,IM AGREE",11.0,NaN,0.25,-2.00,S
site.05.subj.CA-0547.lab.CO-11838-18.iso.1,LEV,UKMYC6,False,05,3.0,HIGH,14,3.0,VZ,3.0,3.0,NaN,"VZ,IM AGREE",11.0,NaN,0.5,-1.00,S
site.03.subj.GB-90840137.lab.IML-01497.iso.1,LEV,UKMYC6,False,03,2.0,LOW,14,2.0,VZ,1.0,5.0,2.0,ALL DISAGREE,11.0,3.0,0.25,-2.00,S


Load metadata from genomes table (lineage and site)

In [75]:
METADATA = pd.read_pickle('data/GENOMES.pkl.gz')
METADATA = METADATA['LINEAGE_NAME']
METADATA

UNIQUEID
site.02.subj.0958.lab.22A197.iso.1                        Lineage 2
site.02.subj.0823.lab.2013241494.iso.1                    Lineage 4
site.02.subj.0359.lab.222018-14.iso.1                     Lineage 2
site.02.subj.0224.lab.2013221088.iso.1                    Lineage 2
site.02.subj.0918.lab.22A153.iso.1                        Lineage 2
                                                            ...    
site.10.subj.YA00023171.lab.YA00023171.iso.1              Lineage 4
site.10.subj.SADH00634109_S16.lab.DH00634109_S16.iso.1             
site.10.subj.H37RV_SA2437.lab.H37RV_SA2437.iso.1                   
site.10.subj.YA00044774.lab.YA00044774.iso.1              Lineage 4
site.10.subj.YA00166043.lab.YA00166043.iso.1              Lineage 2
Name: LINEAGE_NAME, Length: 77860, dtype: category
Categories (7, object): ['', 'Lineage 1', 'Lineage 2', 'Lineage 3', 'Lineage 4', 'Lineage 5', 'Lineage 6']

Load mutation data for gyrA/B genes

In [76]:
MUTATIONS = pd.read_pickle('data/MUTATIONS_SAMPLE.pkl.gz')
MUTATIONS = MUTATIONS.reset_index()
MUTATIONS.set_index('UNIQUEID', inplace = True)

#R463L is a known phylogenetic marker for Beijing lineage
MUTATIONS = MUTATIONS[(~MUTATIONS.IS_NULL) & (MUTATIONS.IS_FILTER_PASS) & (~MUTATIONS.IS_HET) & (MUTATIONS.IS_SNP) \
                    & (MUTATIONS.IN_CDS) & (MUTATIONS.GENE == 'katG') & (MUTATIONS.IS_NONSYNONYMOUS) & (~(MUTATIONS.MUTATION == 'R463L'))]
MUTATIONS

,GENE,MUTATION,POSITION,AMINO_ACID_NUMBER,GENOME_INDEX,NUCLEOTIDE_NUMBER,REF,ALT,IS_SNP,IS_INDEL,...,IS_HET,IS_NULL,IS_FILTER_PASS,ELEMENT_TYPE,MUTATION_TYPE,INDEL_LENGTH,INDEL_1,INDEL_2,SITEID,NUMBER_NUCLEOTIDE_CHANGES
UNIQUEID,,,,,,,,,,,,,,,,,,,,,
site.02.subj.0122.lab.22A076.iso.1,katG,P232A,232.0,232.0,NaN,NaN,ccg,gcg,True,False,...,False,False,True,GENE,AAM,NaN,,,02,1
site.05.subj.LR-3059.lab.CR-00865-16.iso.1,katG,S315T,315.0,315.0,NaN,NaN,agc,acc,True,False,...,False,False,True,GENE,AAM,NaN,,,05,1
site.05.subj.LR-2342.lab.FN-01704-18.iso.1,katG,S315T,315.0,315.0,NaN,NaN,agc,acc,True,False,...,False,False,True,GENE,AAM,NaN,,,05,1
site.05.subj.LI2163509.lab.14722_6_94.iso.1,katG,S315T,315.0,315.0,NaN,NaN,agc,acc,True,False,...,False,False,True,GENE,AAM,NaN,,,05,1
site.05.subj.LI2149109.lab.14893_2_65.iso.1,katG,S315T,315.0,315.0,NaN,NaN,agc,acc,True,False,...,False,False,True,GENE,AAM,NaN,,,05,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
site.10.subj.YA00127822.lab.YA00127822.iso.1,katG,S315T,315.0,315.0,NaN,NaN,agc,acc,True,False,...,False,False,True,GENE,AAM,NaN,,,10,1
site.10.subj.PL00350106.lab.PL00350106.iso.1,katG,S315T,315.0,315.0,NaN,NaN,agc,acc,True,False,...,False,False,True,GENE,AAM,NaN,,,10,1
site.10.subj.YA00024313.lab.YA00024313.iso.1,katG,S315T,315.0,315.0,NaN,NaN,agc,acc,True,False,...,False,False,True,GENE,AAM,NaN,,,10,1


In [77]:
#read in mutations (pre-filtered for gyrA/B mutations due to size of mutations data file)
MUTATIONS = pd.read_csv('data/MUTATIONS-gyrA_B.csv.gz', compression='gzip', header = None)

#restore header through shortened mutations file
HEADER = pd.read_pickle('data/MUTATIONS_SAMPLE.pkl.gz', compression='gzip')

HEADER.reset_index(inplace = True)
HEADER = HEADER.columns.to_list()
MUTATIONS.columns = HEADER

#remove non-synonymous, promoter and mutations with inconclusive sequencing, as well as indels
MUTATIONS = MUTATIONS[(~MUTATIONS.IS_SYNONYMOUS) & (~MUTATIONS.IN_PROMOTER) & (MUTATIONS.IS_SNP) & (~MUTATIONS.IS_NULL) & (MUTATIONS.IS_FILTER_PASS)]
MUTATIONS['GENE_MUTATION'] = MUTATIONS['GENE'] + '_' + MUTATIONS['MUTATION']

#label lineage mutations
#ref for lineage mutations: Miotto ERJ 2017
LINEAGE_MUTATIONS=['gyrA_S95T','gyrA_E21Q','gyrA_T80A','gyrA_A90G','gyrA_G247S','gyrA_A384V',
             'gyrA_G668D','gyrA_L712V', 'gyrA_S250A','gyrA_R252L','gyrA_L398F','gyrA_A463S','gyrA_D639A','gyrA_V742L',
              'gyrB_V301L','gyrB_M291I','gyrB_A403S']
MUTATIONS['LINEAGE_MUTATION'] = MUTATIONS.GENE_MUTATION.isin(LINEAGE_MUTATIONS)

#record how many mutations we see for each sample per UNIQUEID -> we can only perform mutation based approach for samples with < 2 non-lineage mutations in pncA
MUTATIONS.set_index('UNIQUEID', inplace = True)

number_mutations = MUTATIONS[MUTATIONS.LINEAGE_MUTATION == False].index.value_counts()
number_mutations.name = 'MUTATION_COUNT'
MUTATIONS = MUTATIONS.join(number_mutations.to_frame())

print("Out of the", len(MUTATIONS[(MUTATIONS.LINEAGE_MUTATION == False)]), 
      "samples with non-lineage mutations in gyrA/B", 
      len(MUTATIONS[(MUTATIONS.LINEAGE_MUTATION == False) & (MUTATIONS.MUTATION_COUNT < 2)]), 
          "show only one such mutation per sample")

Out of the 5063 samples with non-lineage mutations in gyrA/B 3974 show only one such mutation per sample


In [78]:
#amount of eligible samples where phenotypic data is available
print("Good-quality Phenotypic data is available for", len(PHENOTYPES[PHENOTYPES.index.isin(MUTATIONS[(MUTATIONS.LINEAGE_MUTATION == False) 
                                                                                         & (MUTATIONS.MUTATION_COUNT < 2) ].index.to_list()) 
                                                                      & (PHENOTYPES.PHENOTYPE_QUALITY!='LOW')].index.unique()),
      "of samples that show only one non-lineage mutation per sample in gyrA/B")

Good-quality Phenotypic data is available for 2441 of samples that show only one non-lineage mutation per sample in gyrA/B


Remove low quality phenotypes and divide data by drug (LEV, MXF)

In [79]:
LEV_PHENOTYPES = PHENOTYPES[(PHENOTYPES.index.isin(MUTATIONS[(MUTATIONS.LINEAGE_MUTATION == False) 
                                           & (MUTATIONS.MUTATION_COUNT < 2)].index.to_list())) 
           & (PHENOTYPES.DRUG == 'LEV') & (PHENOTYPES.PHENOTYPE_QUALITY!='LOW')]
LEV_PHENOTYPES.BINARY_PHENOTYPE.value_counts()

BINARY_PHENOTYPE
R    1236
S     903
I       0
Name: count, dtype: int64

In [80]:
MXF_PHENOTYPES = PHENOTYPES[(PHENOTYPES.index.isin(MUTATIONS[(MUTATIONS.LINEAGE_MUTATION == False) 
                                           & (MUTATIONS.MUTATION_COUNT < 2)].index.to_list())) 
           & (PHENOTYPES.DRUG == 'MXF') & (PHENOTYPES.PHENOTYPE_QUALITY!='LOW')]
MXF_PHENOTYPES.BINARY_PHENOTYPE.value_counts()

BINARY_PHENOTYPE
R    1031
S     981
I       0
Name: count, dtype: int64

Join the mutations data with the phenotypic data frame for both drugs

In [81]:
MUTATIONS = MUTATIONS[(MUTATIONS.LINEAGE_MUTATION == False) & (MUTATIONS.MUTATION_COUNT < 2)]

LEV_DATASET = pd.merge(LEV_PHENOTYPES, MUTATIONS, how = 'left', on = 'UNIQUEID')
#LEV_DATASET = pd.merge(LEV_DATASET, METADATA, how = 'left', on = 'UNIQUEID')
LEV_DATASET = LEV_DATASET[['DRUG', 'MIC', 'LOG2MIC', 'BINARY_PHENOTYPE', 'PHENOTYPE_QUALITY', 'GENE', 'MUTATION', 'POSITION', 'GENE_MUTATION']]
LEV_DATASET

,DRUG,MIC,LOG2MIC,BINARY_PHENOTYPE,PHENOTYPE_QUALITY,GENE,MUTATION,POSITION,GENE_MUTATION
UNIQUEID,,,,,,,,,
site.06.subj.06TB_0027.lab.06MIL0488.iso.1,LEV,2.0,1.0,R,MEDIUM,gyrA,A90V,90.0,gyrA_A90V
site.06.subj.06TB_0843.lab.06MIL1790.iso.1,LEV,>8,4.0,R,HIGH,gyrA,D94Y,94.0,gyrA_D94Y
site.04.subj.00624.lab.709302.iso.1,LEV,>8,4.0,R,HIGH,gyrA,D94N,94.0,gyrA_D94N
site.11.subj.XTB-18-196.lab.XTB-18-196.iso.1,LEV,2.0,1.0,R,MEDIUM,gyrB,N499D,499.0,gyrB_N499D
site.10.subj.YA00021495.lab.YA00021495.iso.1,LEV,0.5,-1.0,S,HIGH,gyrA,N193S,193.0,gyrA_N193S
...,...,...,...,...,...,...,...,...,...
site.10.subj.XD02435891.lab.XD02435891.iso.1,LEV,8.0,3.0,R,HIGH,gyrA,D94N,94.0,gyrA_D94N
site.02.subj.0482.lab.235081-14.iso.1,LEV,1.0,0.0,S,HIGH,gyrB,G512R,512.0,gyrB_G512R
site.05.subj.LR-2321.lab.FN-01077-18.iso.1,LEV,>8,4.0,R,HIGH,gyrA,G88C,88.0,gyrA_G88C


In [82]:
MXF_DATASET = pd.merge(MXF_PHENOTYPES, MUTATIONS, how = 'left', on = 'UNIQUEID')
#MXF_DATASET = pd.merge(MXF_DATASET, METADATA, how = 'left', on = 'UNIQUEID')
MXF_DATASET = MXF_DATASET[['DRUG', 'MIC', 'LOG2MIC', 'BINARY_PHENOTYPE', 'PHENOTYPE_QUALITY', 'GENE', 'MUTATION', 'POSITION', 'GENE_MUTATION']]
MXF_DATASET

,DRUG,MIC,LOG2MIC,BINARY_PHENOTYPE,PHENOTYPE_QUALITY,GENE,MUTATION,POSITION,GENE_MUTATION
UNIQUEID,,,,,,,,,
site.04.subj.00097.lab.633433.iso.1,MXF,2.0,1.00,R,HIGH,gyrA,D94G,94.0,gyrA_D94G
site.10.subj.UH00935117.lab.UH00935117.iso.1,MXF,4.0,2.00,R,HIGH,gyrA,A90V,90.0,gyrA_A90V
site.11.subj.XTB-18-164.lab.XTB-18-164.iso.1,MXF,>4,3.00,R,MEDIUM,gyrA,D94N,94.0,gyrA_D94N
site.04.subj.00121.lab.633619.iso.1,MXF,2.0,1.00,R,HIGH,gyrA,D94G,94.0,gyrA_D94G
site.14.subj.2591.lab.2591.iso.1,MXF,>4,3.00,R,HIGH,gyrA,D94N,94.0,gyrA_D94N
...,...,...,...,...,...,...,...,...,...
site.04.subj.01310.lab.724468.iso.1,MXF,1.0,0.00,S,HIGH,gyrA,S91P,91.0,gyrA_S91P
site.11.subj.XTB-18-186.lab.XTB-18-186.iso.1,MXF,2.0,1.00,R,MEDIUM,gyrA,A90V,90.0,gyrA_A90V
site.10.subj.UH01302806.lab.UH01302806.iso.1,MXF,2.0,1.00,R,HIGH,gyrA,A90V,90.0,gyrA_A90V


Create features using sbml core

In [83]:
def make_segid(row):
    return row['GENE'][3:]

MXF_DATASET['segid'] = MXF_DATASET.apply(make_segid, axis = 1)
MXF_DATASET['mutation'] = MXF_DATASET['MUTATION']
MXF_DATASET.drop('MUTATION', axis = 1, inplace = True)

LEV_DATASET['segid'] = LEV_DATASET.apply(make_segid, axis = 1)
LEV_DATASET['mutation'] = LEV_DATASET['MUTATION']
LEV_DATASET.drop('MUTATION', axis = 1, inplace = True)

In [84]:
MXF_DATASET

,DRUG,MIC,LOG2MIC,BINARY_PHENOTYPE,PHENOTYPE_QUALITY,GENE,POSITION,GENE_MUTATION,segid,mutation
UNIQUEID,,,,,,,,,,
site.04.subj.00097.lab.633433.iso.1,MXF,2.0,1.00,R,HIGH,gyrA,94.0,gyrA_D94G,A,D94G
site.10.subj.UH00935117.lab.UH00935117.iso.1,MXF,4.0,2.00,R,HIGH,gyrA,90.0,gyrA_A90V,A,A90V
site.11.subj.XTB-18-164.lab.XTB-18-164.iso.1,MXF,>4,3.00,R,MEDIUM,gyrA,94.0,gyrA_D94N,A,D94N
site.04.subj.00121.lab.633619.iso.1,MXF,2.0,1.00,R,HIGH,gyrA,94.0,gyrA_D94G,A,D94G
site.14.subj.2591.lab.2591.iso.1,MXF,>4,3.00,R,HIGH,gyrA,94.0,gyrA_D94N,A,D94N
...,...,...,...,...,...,...,...,...,...,...
site.04.subj.01310.lab.724468.iso.1,MXF,1.0,0.00,S,HIGH,gyrA,91.0,gyrA_S91P,A,S91P
site.11.subj.XTB-18-186.lab.XTB-18-186.iso.1,MXF,2.0,1.00,R,MEDIUM,gyrA,90.0,gyrA_A90V,A,A90V
site.10.subj.UH01302806.lab.UH01302806.iso.1,MXF,2.0,1.00,R,HIGH,gyrA,90.0,gyrA_A90V,A,A90V


In [85]:
def update_binary_phenotype(row, dataset):
        
    target_string = row['GENE_MUTATION']
    
    filtered_df = dataset[dataset['GENE_MUTATION'] == target_string]
    
    if len(filtered_df) < 2:
        
        return pd.Series([row['BINARY_PHENOTYPE'], 1.0])
    
    else:
    
        most_common_value = filtered_df['BINARY_PHENOTYPE'].mode().iloc[0]
        label_support = filtered_df.BINARY_PHENOTYPE.value_counts()[most_common_value]/sum(filtered_df.BINARY_PHENOTYPE.value_counts())

        return pd.Series([most_common_value, label_support])
    

In [86]:
MXF_DATASET[['CLEAN_BINARY_PHENOTYPE', 'LABEL_SUPPORT']] = MXF_DATASET.apply(lambda row: update_binary_phenotype(row, MXF_DATASET), axis=1)

In [87]:
MXF_DATASET

,DRUG,MIC,LOG2MIC,BINARY_PHENOTYPE,PHENOTYPE_QUALITY,GENE,POSITION,GENE_MUTATION,segid,mutation,CLEAN_BINARY_PHENOTYPE,LABEL_SUPPORT
UNIQUEID,,,,,,,,,,,,
site.04.subj.00097.lab.633433.iso.1,MXF,2.0,1.00,R,HIGH,gyrA,94.0,gyrA_D94G,A,D94G,R,0.818923
site.10.subj.UH00935117.lab.UH00935117.iso.1,MXF,4.0,2.00,R,HIGH,gyrA,90.0,gyrA_A90V,A,A90V,R,0.555556
site.11.subj.XTB-18-164.lab.XTB-18-164.iso.1,MXF,>4,3.00,R,MEDIUM,gyrA,94.0,gyrA_D94N,A,D94N,R,0.823529
site.04.subj.00121.lab.633619.iso.1,MXF,2.0,1.00,R,HIGH,gyrA,94.0,gyrA_D94G,A,D94G,R,0.818923
site.14.subj.2591.lab.2591.iso.1,MXF,>4,3.00,R,HIGH,gyrA,94.0,gyrA_D94N,A,D94N,R,0.823529
...,...,...,...,...,...,...,...,...,...,...,...,...
site.04.subj.01310.lab.724468.iso.1,MXF,1.0,0.00,S,HIGH,gyrA,91.0,gyrA_S91P,A,S91P,R,0.676923
site.11.subj.XTB-18-186.lab.XTB-18-186.iso.1,MXF,2.0,1.00,R,MEDIUM,gyrA,90.0,gyrA_A90V,A,A90V,R,0.555556
site.10.subj.UH01302806.lab.UH01302806.iso.1,MXF,2.0,1.00,R,HIGH,gyrA,90.0,gyrA_A90V,A,A90V,R,0.555556


In [88]:
LEV_DATASET[['CLEAN_BINARY_PHENOTYPE', 'LABEL_SUPPORT']] = LEV_DATASET.apply(lambda row: update_binary_phenotype(row, LEV_DATASET), axis=1)

Drop rows with label support under 75%

In [89]:
MXF_DATASET = MXF_DATASET[MXF_DATASET.LABEL_SUPPORT > 0.50]
LEV_DATASET = LEV_DATASET[LEV_DATASET.LABEL_SUPPORT > 0.50]

For now, drop rows with mutations outside of crystallized regions, might impute values later on (segid A: 15-501, segid B: 425-675)

In [90]:
MXF_DATASET = MXF_DATASET[(MXF_DATASET.segid == 'A') & MXF_DATASET.POSITION.astype(int).isin(list(range(15,502))) 
            | (MXF_DATASET.segid == 'B') & MXF_DATASET.POSITION.astype(int).isin(list(range(425,676)))]

LEV_DATASET = LEV_DATASET[(LEV_DATASET.segid == 'A') & LEV_DATASET.POSITION.astype(int).isin(list(range(15,502))) 
            | (LEV_DATASET.segid == 'B') & LEV_DATASET.POSITION.astype(int).isin(list(range(425,676)))]

In [91]:
import MDAnalysis as mda

pdb = mda.Universe('data/pdb/MFX/5bs8.pdb')

In [92]:
protein = pdb.select_atoms("protein")
protein
#protein.write("data/5bs8_atoms.pdb") #can change this to 

<AtomGroup with 23125 atoms>

In [93]:
snap2 = pd.read_csv('data/SNAP2_complete.csv')
snap2

,Variant,Predicted Effect,Score,Expected Accuracy,segid
0,T1A,effect,25,63%,A
1,T1R,effect,57,75%,A
2,T1N,neutral,-80,87%,A
3,T1D,effect,21,63%,A
4,T1C,effect,13,59%,A
...,...,...,...,...,...
15115,V253S,effect,52,75%,B
15116,V253T,effect,30,66%,B
15117,V253W,effect,66,80%,B
15118,V253Y,effect,56,75%,B


In [94]:
def correct_position(row):
    
    if row.segid == 'A':
        increase_by = 1
    else:
        increase_by = 422

    letter1 = row.Variant[0]
    letter2 = row.Variant[-1]  # Extract the letters (all characters except the last one)
    number = int(row.Variant[1:-1])  # Extract the number and convert it to an integer
    updated_number = number + increase_by  # Increase the number by the specified amount
    updated_string = f'{letter1}{updated_number}{letter2}'  # Create the updated string

    return updated_string

In [95]:
snap2['Variant'] = snap2.apply(correct_position, axis = 1)
snap2.to_csv('data/SNAP2_complete_clean.csv', index=False)

In [96]:
data_dict = {}
names = ['MXF_df', 'LEV_df']
i=0

for dataset in [MXF_DATASET, LEV_DATASET]:

    sbml_dataset = dataset[['segid', 'mutation']]
    sbml_dataset.drop_duplicates(subset='mutation', keep='first', inplace=True)

    features = sbmlcore.FeatureDataset(sbml_dataset, species='M. tuberculosis', protein = 'DNA Gyrase')

    a = sbmlcore.AminoAcidVolumeChange()
    b1 = sbmlcore.AminoAcidHydropathyChangeKyteDoolittle()
    b2 = sbmlcore.AminoAcidHydropathyChangeWimleyWhite()
    c = sbmlcore.AminoAcidMWChange()
    d = sbmlcore.AminoAcidPiChange()
    e = sbmlcore.Stride('data/pdb/5bs8_atoms.pdb')
    f = sbmlcore.FreeSASA('data/pdb/5bs8_atoms.pdb')
    g = sbmlcore.SNAP2("data/SNAP2_complete_clean.csv", offsets = {'A': 0, 'B': 0})

    print(names[i])
    if names[i] == 'MXF_df':

        h1 = sbmlcore.StructuralDistances('data/pdb/MFX/5bs8_no_PTR_su_AB_MGB.pdb', distance_selection="resname MG", distance_name='dist_MGB')
        h2 = sbmlcore.StructuralDistances('data/pdb/MFX/5bs8_no_PTR_su_AB_MGE.pdb', distance_selection="resname MG", distance_name='dist_MGE')
        # h3 = sbmlcore.StructuralDistances('data/pdb/5bs8_no_PTR_su_AB_MGE.pdb', distance_selection="resname MG", distance_name='dist_MGE')
        # h4 = sbmlcore.StructuralDistances('data/pdb/5bs8_no_PTR_su_AB_MGF.pdb', distance_selection="resname MG", distance_name='dist_MGF')
        k = sbmlcore.StructuralDistances('data/pdb/MFX/5bs8_no_PTR_su_AB_MFX_H.pdb', distance_selection="resname MFX", distance_name='dist_drug')

    else:

        h1 = sbmlcore.StructuralDistances('data/pdb/LEV/5btg_no_PTR_su_AB_MGB.pdb', distance_selection="resname MG", distance_name='dist_MGB')
        h2 = sbmlcore.StructuralDistances('data/pdb/LEV/5btg_no_PTR_su_AB_MGE.pdb', distance_selection="resname MG", distance_name='dist_MGE')
        # h3 = sbmlcore.StructuralDistances('data/pdb/5btg_no_PTR_su_AB_MGE.pdb', distance_selection="resname MG", distance_name='dist_MGE')
        # h4 = sbmlcore.StructuralDistances('data/pdb/5btg_no_PTR_su_AB_MGF.pdb', distance_selection="resname MG", distance_name='dist_MGF')
        k = sbmlcore.StructuralDistances('data/pdb/LEV/5btg_no_PTR_su_AB_LEV_E.pdb', distance_selection="resname LFX", distance_name='dist_drug')

    features.add_feature([a, b1, b2, c, d, e, f, g, h1, h2, k])
    features = features.df

    dataset.reset_index(inplace = True)

    variable_name = names[i]
    i = i+1
    
    data_dict[variable_name] = pd.merge(dataset, features, how='left', on=['segid','mutation'])

MXF_df
LEV_df


In [97]:
df = data_dict['MXF_df']
df.columns

Index(['UNIQUEID', 'DRUG', 'MIC', 'LOG2MIC', 'BINARY_PHENOTYPE',
       'PHENOTYPE_QUALITY', 'GENE', 'POSITION', 'GENE_MUTATION', 'segid',
       'mutation', 'CLEAN_BINARY_PHENOTYPE', 'LABEL_SUPPORT', 'd_volume',
       'd_hydropathy_KD', 'd_hydropathy_WW', 'd_MW', 'd_Pi',
       'secondary_structure', 'secondary_structure_long', 'phi', 'psi',
       'residue_sasa', 'n_hbond_acceptors', 'n_hbond_donors', 'B', 'C', 'E',
       'G', 'H', 'T', 'SASA', 'snap2_score', 'snap2_accuracy', 'dist_MGB',
       'dist_MGE', 'dist_drug'],
      dtype='object')

In [438]:
# df.drop_duplicates(subset=['GENE_MUTATION','LINEAGE_NAME'], keep='first', inplace=True)
# df.GENE_MUTATION.value_counts()[:20]

In [109]:
def replace_symbol(input_string, target_symbol, replacement_symbol):
   
    modified_string = input_string.replace(target_symbol, replacement_symbol)
    
    return modified_string

In [110]:
df.GENE_MUTATION.unique()

array(['gyrA_D94G', 'gyrA_A90V', 'gyrA_D94N', 'gyrA_D94A', 'gyrB_T500N',
       'gyrA_D94Y', 'gyrA_S91P', 'gyrA_D94H', 'gyrB_E501D', 'gyrB_K526Q',
       'gyrB_S447Y', 'gyrA_N193S', 'gyrB_R575H', 'gyrA_V223I',
       'gyrA_I362M', 'gyrB_K668E', 'gyrA_G236V', 'gyrA_K319N',
       'gyrA_D488E', 'gyrB_R446C', 'gyrA_I427V', 'gyrA_G88C',
       'gyrB_M530T', 'gyrB_N499I', 'gyrB_D494A', 'gyrB_P439R',
       'gyrA_A456V', 'gyrA_T377I', 'gyrA_A288V', 'gyrA_A406T',
       'gyrB_R446H', 'gyrB_S661T', 'gyrA_P472S', 'gyrA_L296P',
       'gyrB_N499D', 'gyrA_L205V', 'gyrA_R252G', 'gyrB_A508V',
       'gyrB_S447F', 'gyrA_L401V', 'gyrA_A322S', 'gyrA_E317G',
       'gyrA_Q22H', 'gyrA_H368Q', 'gyrA_G477E', 'gyrB_F549L',
       'gyrA_D481G', 'gyrA_G88A', 'gyrA_L465Q', 'gyrA_T267I',
       'gyrB_D461N', 'gyrA_S237P', 'gyrB_S576T', 'gyrA_R254C',
       'gyrA_E140G', 'gyrA_L346V', 'gyrB_K603R', 'gyrB_V630I',
       'gyrB_S576G', 'gyrA_P154R', 'gyrB_T664I', 'gyrA_L105R',
       'gyrB_A504V', 'gyrA_D89N', 'gy

In [111]:
WHO_list = WHO_data[WHO_data.DRUG == 'MXF'].MUTATION.unique()
WHO_list
WHO_list = [replace_symbol(s, '@', '_') for s in WHO_list]

In [116]:
found_strings = [string for string in WHO_list if string not in df.GENE_MUTATION.unique()]

found_strings

['gyrB_E501V',
 'gyrA_A384V',
 'gyrA_A463S',
 'gyrA_A90G',
 'gyrA_E21Q',
 'gyrA_G247S',
 'gyrA_G668D',
 'gyrA_S95T',
 'gyrA_T80A',
 'gyrA_c-34t',
 'gyrB_M291I',
 'gyrB_P94L',
 'gyrB_V301L',
 'gyrB_c-165t',
 'gyrA_*=',
 'gyrB_*=']

In [440]:
g = sbmlcore.TempFactors('data/5bs8.pdb')
g

AssertionError: File does not exist!

Further features to be added soon (DeepDG)

In [441]:
for dataset in ['LEV_df', 'MXF_df']:

    df = data_dict[dataset]
    print(df.info())
    
    name = f'data/ML_DATA_{dataset[:3]}.csv'
    df.to_csv(name, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1723 entries, 0 to 1722
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   UNIQUEID                  1723 non-null   object  
 1   DRUG                      1723 non-null   category
 2   MIC                       1710 non-null   category
 3   LOG2MIC                   1710 non-null   float64 
 4   BINARY_PHENOTYPE          1710 non-null   category
 5   PHENOTYPE_QUALITY         1723 non-null   category
 6   GENE                      1723 non-null   object  
 7   POSITION                  1723 non-null   float64 
 8   GENE_MUTATION             1723 non-null   object  
 9   segid                     1723 non-null   object  
 10  mutation                  1723 non-null   object  
 11  CLEAN_BINARY_PHENOTYPE    1722 non-null   object  
 12  LABEL_SUPPORT             1723 non-null   float64 
 13  d_volume                  1723 non-null   float6

In [442]:
df

,UNIQUEID,DRUG,MIC,LOG2MIC,BINARY_PHENOTYPE,PHENOTYPE_QUALITY,GENE,POSITION,GENE_MUTATION,segid,...,E,G,H,T,SASA,snap2_score,snap2_accuracy,dist_MGB,dist_MGE,dist_drug
0,site.04.subj.00097.lab.633433.iso.1,MXF,2.0,1.00,R,HIGH,gyrA,94.0,gyrA_D94G,A,...,False,False,True,False,88.155089,63,80,18.494689,7.838828,14.503318
1,site.10.subj.UH00935117.lab.UH00935117.iso.1,MXF,4.0,2.00,R,HIGH,gyrA,90.0,gyrA_A90V,A,...,False,False,True,False,55.555816,-64,82,14.434102,5.483332,10.609526
2,site.11.subj.XTB-18-164.lab.XTB-18-164.iso.1,MXF,>4,3.00,R,MEDIUM,gyrA,94.0,gyrA_D94N,A,...,False,False,True,False,88.155089,56,75,18.494689,7.838828,14.503318
3,site.04.subj.00121.lab.633619.iso.1,MXF,2.0,1.00,R,HIGH,gyrA,94.0,gyrA_D94G,A,...,False,False,True,False,88.155089,63,80,18.494689,7.838828,14.503318
4,site.14.subj.2591.lab.2591.iso.1,MXF,>4,3.00,R,HIGH,gyrA,94.0,gyrA_D94N,A,...,False,False,True,False,88.155089,56,75,18.494689,7.838828,14.503318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660,site.04.subj.01310.lab.724468.iso.1,MXF,1.0,0.00,S,HIGH,gyrA,91.0,gyrA_S91P,A,...,False,False,True,False,65.228155,63,80,13.712391,5.381126,11.010229
1661,site.11.subj.XTB-18-186.lab.XTB-18-186.iso.1,MXF,2.0,1.00,R,MEDIUM,gyrA,90.0,gyrA_A90V,A,...,False,False,True,False,55.555816,-64,82,14.434102,5.483332,10.609526
1662,site.10.subj.UH01302806.lab.UH01302806.iso.1,MXF,2.0,1.00,R,HIGH,gyrA,90.0,gyrA_A90V,A,...,False,False,True,False,55.555816,-64,82,14.434102,5.483332,10.609526
1663,site.05.subj.CA-0020.lab.CO-05421-18.iso.1,MXF,0.12,-3.06,S,HIGH,gyrB,594.0,gyrB_G594R,B,...,False,False,True,False,2.551348,33,66,29.300727,43.081517,41.932227
